In [51]:
import numpy as np

In [52]:
from Actfunc import*
from init import*
from Loss import*

In [53]:
Relu(np.array([-2,0,3]))

array([0., 0., 3.])

In [ ]:
class Layer:

    def __init__(self,dim_in,dim_out,init_fn,act_fn=Relu):
        self.weights=init_fn(dim_in,dim_out)
        self.bias=np.reshape(init_fn(1,dim_out),-1)
        self.act_fn=act_fn


    def fw(self,input):
        output=self.weights@input + self.bias
        Z=output
        if self.act_fn:
            output=self.act_fn(Z)
        return output,Z

    def gradient_last(self,onehot,A,Z):
        dc_da_j=softmax(Z[-1])
        tæller=np.sum(np.array([-np.exp(Z[-1][i]) for i in range(len(Z[-1])) if i!=int(np.where(onehot==1)[0][0])]))
        tæller/=np.sum(np.exp(Z[-1]))
        dc_da_j[np.where(onehot==1)[0][0]]=tæller
        gradient_w=np.array([A[-1]*dc_da for dc_da in dc_da_j ])
        gradient_b=dc_da_j
        return gradient_w,gradient_b,dc_da_j
    
    def gradient_not_last(self,onehot,a_in_front,da_in_front,z,z_in_front,w_in_front,d_actfun):
        

    def gradient_last(self,onehot,A,Z):
        y_predicted = softmax(Z[-1])
        d = y_predicted - onehot # not sure baout this
        a_prev = A[-2] # previos layr
        gradient_w = np.outer(d,a_prev) # matrice ud
        gradient_b = d
        return gradient_w,gradient_b, d #den her skal genbruges?
        


            
            




In [ ]:
class FFNN:

    def __init__(self,input_size,hidden_sizes,output_size, init_fn,act_fn):
        self.layers=[]
        self.layers.append(Layer(dim_in=input_size,dim_out=hidden_sizes[0],init_fn=init_fn,act_fn=act_fn))
        for i in range(1,len(hidden_sizes)):
            self.layers.append(Layer(dim_in=hidden_sizes[i-1],dim_out=hidden_sizes[i],init_fn=init_fn,act_fn=act_fn))
        self.layers.append(Layer(dim_in=hidden_sizes[-1],dim_out=output_size,init_fn=init_fn,act_fn=False))

    

    def forward(self,input):

        A=[]
        Z=[]
        output=input
        for layer in self.layers:
            a,z=layer.fw(output)
            A.append(a)
            Z.append(z)
            output=a
        return output,A,Z
    

    

    def print_info(self):
        for layer in self.layers:
            print("weights")
            print(layer.weights)
            print("bias")
            print(layer.bias)





In [56]:
test=FFNN(input_size=3,hidden_sizes=[2,2,4,4],output_size=3,init_fn=xavier_uniform,act_fn=Relu)

In [57]:
test_input=np.array([3,14,5])


In [58]:
test_output,A,Z=test.forward(test_input)

In [59]:
print(test_output)

[-0.97104247  0.10234346 -0.29879336]


In [60]:
print(A)

[array([19.44277457,  0.        ]), array([5.47482011, 0.        ]), array([0.        , 0.        , 4.85768663, 0.        ]), array([0.        , 0.        , 0.        , 2.56077487]), array([-0.97104247,  0.10234346, -0.29879336])]


In [61]:
print(Z)

[array([19.44277457, -5.87438628]), array([  5.47482011, -12.84747794]), array([-4.49034647, -4.5185205 ,  4.85768663, -0.31126228]), array([-3.81960191, -1.95638476, -2.32911333,  2.56077487]), array([-0.97104247,  0.10234346, -0.29879336])]
